## Simple Way to Convert Text to Numbers

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Define a sample text
text1 = "The sun was setting over the horizon, casting a warm orange glow across the sky. The waves gently lapped at the shore as the seagulls flew overhead."
text2 = "As the night fell, the stars began to twinkle in the sky. The sound of crickets filled the air as the fireflies danced among the trees."

In [ ]:
vectorizer = TfidfVectorizer()
print(vectorizer.fit_transform([text1, text2]).toarray())

[[0.16387439 0.         0.         0.11659798 0.16387439 0.
  0.16387439 0.         0.         0.         0.         0.
  0.16387439 0.16387439 0.16387439 0.16387439 0.         0.16387439
  0.         0.         0.16387439 0.16387439 0.16387439 0.16387439
  0.16387439 0.16387439 0.11659798 0.         0.         0.16387439
  0.69958785 0.         0.         0.         0.16387439 0.16387439
  0.16387439]
 [0.         0.15190417 0.15190417 0.21616214 0.         0.15190417
  0.         0.15190417 0.15190417 0.15190417 0.15190417 0.15190417
  0.         0.         0.         0.         0.15190417 0.
  0.15190417 0.15190417 0.         0.         0.         0.
  0.         0.         0.10808107 0.15190417 0.15190417 0.
  0.75656749 0.15190417 0.15190417 0.15190417 0.         0.
  0.        ]]


### Using Vector DB & Langchain

#### Install Libraries

In [ ]:
!pip install elasticsearch langchain openai tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.9/387.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.6/727.6 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

#### Setup Open AI API key

In [ ]:
import os


In [ ]:
os.environ['OPENAI_API_KEY'] = 'PASTE YOUR KEY HERE'

#### Setup Elasticsearch Credentials

In [ ]:
from langchain import ElasticVectorSearch
from langchain.embeddings import OpenAIEmbeddings

embedding = OpenAIEmbeddings()

elastic_host = "ES_HOST"
username = "ES_USER"
password = "ES_PASS"
elasticsearch_url = f"https://{username}:{password}@{elastic_host}:9243"

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch
from langchain.document_loaders import TextLoader

#### Load data and perform Vector Search on ES Data

In [ ]:
from langchain.document_loaders import TextLoader
loader = TextLoader('./sample_data/state_of_the_union.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

#### Get Similarity Search Results

In [ ]:
db = ElasticVectorSearch.from_documents(docs, embeddings, elasticsearch_url=elasticsearch_url)

query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

In [ ]:
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
